In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

In [2]:
df = pd.read_csv('/content/drive/MyDrive/TOBIGS/7주차/spam.csv')

In [3]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


---

- NLTK(Natural Language Toolkit): 영어의 토크나이징에 자주 사용되며, 다양한 언어 처리 도구를 제공  
- Spacy: NLTK에 비해 더 빠른 성능을 제공하며, 여러 형태의 토큰화 수행이 가능

In [4]:
# NLTK 리소스 다운로드
nltk.download('punkt')
nltk.download('stopwords')

# 텍스트 전처리 함수
def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    # 불용어 및 특수 문자 제거
    tokens = [word for word in tokens if word not in stopwords.words('english') and word not in string.punctuation]
    return tokens

# 전처리 수행
df['v3'] = df['v2'].apply(preprocess_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
import spacy
nlp = spacy.load('en_core_web_sm')
df['v4'] = df['v2'].apply(lambda text: [token.text for token in nlp(text)])

In [ ]:
df

,v1,v2,v3,v4
0,ham,"Go until jurong point, crazy.. Available only ...","[go, jurong, point, crazy, .., available, bugi...","[Go, until, jurong, point, ,, crazy, .., Avail..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, ..., joking, wif, u, oni, ...]","[Ok, lar, ..., Joking, wif, u, oni, ...]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, 2, wkly, comp, win, fa, cup, fin...","[Free, entry, in, 2, a, wkly, comp, to, win, F..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, early, hor, ..., u, c, already, ...","[U, dun, say, so, early, hor, ..., U, c, alrea..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, n't, think, goes, usf, lives, around, th...","[Nah, I, do, n't, think, he, goes, to, usf, ,,..."
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,"[2nd, time, tried, 2, contact, u., u, å£750, p...","[This, is, the, 2nd, time, we, have, tried, 2,..."
5568,ham,Will Ì_ b going to esplanade fr home?,"[ì_, b, going, esplanade, fr, home]","[Will, Ì, _, b, going, to, esplanade, fr, home..."
5569,ham,"Pity, * was in mood for that. So...any other s...","[pity, mood, ..., suggestions]","[Pity, ,, *, was, in, mood, for, that, ., So, ..."
5570,ham,The guy did some bitching but I acted like i'd...,"[guy, bitching, acted, like, 'd, interested, b...","[The, guy, did, some, bitching, but, I, acted,..."


---

In [ ]:
import gensim.downloader as api

# 300차원 벡터를 사용하는 사전학습된 임베딩 사용
word2vec = api.load('word2vec-google-news-300')
glove = api.load('glove-wiki-gigaword-300')
fasttext = api.load('fasttext-wiki-news-subwords-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded
[==================================================] 100.0% 376.1/376.1MB downloaded
[==================================================] 100.0% 958.5/958.4MB downloaded


- CountVectorizer: 단어의 빈도를 기준으로 텍스트 벡터를 생성
- TfidfVectorizer: TF-IDF(Term Frequency-Inverse Document Frequency)를 사용하여 중요한 단어에 더 많은 가중치를 주는 방식  
- Word2Vec의 CBOW(Continuous Bag-of-Words): 주변 단어들을 통해 특정 단어를 예측하는 방식으로 벡터를 생성  
- Word2Vec의 Skip-Gram: 특정 단어를 통해 주변 단어들을 예측하는 방식으로 벡터를 생성  
- GloVe (Global Vectors for Word Representation): GloVe는 전역 단어 공동 출현 통계와 지역적 맥락 정보를 결합해 단어 벡터를 생성  
- FastText: Facebook에 의해 개발된 이 기법은 서브워드 정보를 활용하여 Word2Vec을 확장한 방식으로 특히, 드문 단어나 오타가 있는 단어에 유용함  
- BERT, GPT (Transformer 기반 모델들): 이들은 문맥에 따라 단어 의미가 변화하는 것을 고려하여 매번 다른 표현을 생성  

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from gensim.models import KeyedVectors

# 평균 임베딩 벡터를 계산하는 함수
def get_average_vector(tokens_list, model_keyed_vectors):
  # tokens_list: 처리하려는 토큰(단어)의 리스트
  #  model_keyed_vectors: 사전학습된 단어 임베딩 모델로부터 가져온 키-벡터 데이터
    vector_list = [model_keyed_vectors[token] for token in tokens_list if token in model_keyed_vectors]
    if len(vector_list) == 0:
        return np.zeros(model_keyed_vectors.vector_size)
    average_vector = np.mean(vector_list, axis=0)
    # vector_list에 벡터가 있을 경우, np.mean 함수를 통해 모든 벡터의 평균을 계산
    return average_vector

    # -> 문장의 의미를 벡터로 표현하기 위해 문장에 있는 모든 단어의 벡터를 평균내는 방식으로 활용

# 모델 테스트를 위한 함수
def test_model(df, model_keyed_vectors, text_column, target_column):
    # 텍스트를 평균 임베딩 벡터로 변환
    X = np.array([get_average_vector(tokens, model_keyed_vectors) for tokens in df[text_column]])
    Y = df[target_column]

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    classifier = LogisticRegression(max_iter=1000)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    # 분류 리포트 생성
    report = classification_report(y_test, y_pred)

    return report



In [ ]:
 test_model(df, word2vec, 'v2', 'v1')

'              precision    recall  f1-score   support\n         ham       0.97      0.99      0.98       965\n        spam       0.94      0.77      0.85       150\n    accuracy                           0.96      1115\n   macro avg       0.95      0.88      0.91      1115\nweighted avg       0.96      0.96      0.96      1115'

In [ ]:
              precision    recall  f1-score   support
         ham       0.97      0.99      0.98       965
        spam       0.94      0.77      0.85       150
    accuracy                           0.96      1115
   macro avg       0.95      0.88      0.91      1115
weighted avg       0.96      0.96      0.96      1115

In [ ]:
 test_model(df, word2vec, 'v3', 'v1')

'              precision    recall  f1-score   support\n         ham       0.96      0.98      0.97       965\n        spam       0.85      0.71      0.78       150\n    accuracy                           0.94      1115\n   macro avg       0.90      0.85      0.87      1115\nweighted avg       0.94      0.94      0.94      1115'

In [ ]:
              precision    recall  f1-score   support
         ham       0.96      0.98      0.97       965
        spam       0.85      0.71      0.78       150
    accuracy                           0.94      1115
   macro avg       0.90      0.85      0.87      1115
weighted avg       0.94      0.94      0.94      1115

In [ ]:
 test_model(df, word2vec, 'v4', 'v1')

'              precision    recall  f1-score   support\n         ham       0.97      0.99      0.98       965\n        spam       0.94      0.77      0.85       150\n    accuracy                           0.96      1115\n   macro avg       0.95      0.88      0.91      1115\nweighted avg       0.96      0.96      0.96      1115'

In [ ]:
              precision    recall  f1-score   support
         ham       0.97      0.99      0.98       965
        spam       0.94      0.77      0.85       150
    accuracy                           0.96      1115
   macro avg       0.95      0.88      0.91      1115
weighted avg       0.96      0.96      0.96      1115

v2와 v4의 결과 높은 정확도를 유지하면서도, 전체 스팸 중 높은 비율(Recall)을 찾아내고 있는 반면, v3는 상대적으로 낮다. 정밀도는 v3에서 Spam 분류에 있어서 더 보수적이거나, 오진을 덜 내는 대신 실제 스팸을 놓치는 경우가 많다. 따라서, v3 모델을 사용할 경우 스팸 식별 능력이 v2나 v4에 비해 다소 제한적일 수 있다.

v2와 v4가 더 좋을지도?

---

In [ ]:
 test_model(df, glove, 'v2', 'v1')

'              precision    recall  f1-score   support\n         ham       0.97      0.99      0.98       965\n        spam       0.96      0.79      0.86       150\n    accuracy                           0.97      1115\n   macro avg       0.96      0.89      0.92      1115\nweighted avg       0.97      0.97      0.97      1115'

In [ ]:
              precision    recall  f1-score   support
         ham       0.97      0.99      0.98       965
        spam       0.96      0.79      0.86       150
    accuracy                           0.97      1115
   macro avg       0.96      0.89      0.92      1115
weighted avg       0.97      0.97      0.97      1115

In [ ]:
 test_model(df, glove, 'v3', 'v1')

'              precision    recall  f1-score   support\n         ham       0.96      0.97      0.97       965\n        spam       0.82      0.77      0.79       150\n    accuracy                           0.95      1115\n   macro avg       0.89      0.87      0.88      1115\nweighted avg       0.94      0.95      0.94      1115'

In [ ]:
              precision    recall  f1-score   support
         ham       0.96      0.97      0.97       965
        spam       0.82      0.77      0.79       150
    accuracy                           0.95      1115
   macro avg       0.89      0.87      0.88      1115
weighted avg       0.94      0.95      0.94      1115

In [ ]:
 test_model(df, glove, 'v4', 'v1')

'              precision    recall  f1-score   support\n         ham       0.96      0.98      0.97       965\n        spam       0.86      0.71      0.78       150\n    accuracy                           0.95      1115\n   macro avg       0.91      0.85      0.87      1115\nweighted avg       0.94      0.95      0.94      1115'

In [ ]:
              precision    recall  f1-score   support
         ham       0.96      0.98      0.97       965
        spam       0.86      0.71      0.78       150
    accuracy                           0.95      1115
   macro avg       0.91      0.85      0.87      1115
weighted avg       0.94      0.95      0.94      1115

성능 면에서 v2는 가장 높은 Precision을 보여 정확한 분류 능력을 가지고 있다. 특히 Spam에 대한 Recall이 0.79로 다른 것에 비해 높은 편이며, 이는 실제 Spam을 잘 식별해내고 있음을 의미한다. 따라서, 일반적인 이메일 분류 작업에서 v2가 더욱 효과적일 것으로 생각된다.

---

In [ ]:
 test_model(df, fasttext, 'v2', 'v1')

'              precision    recall  f1-score   support\n         ham       0.95      1.00      0.97       965\n        spam       0.96      0.67      0.79       150\n    accuracy                           0.95      1115\n   macro avg       0.96      0.83      0.88      1115\nweighted avg       0.95      0.95      0.95      1115'

In [ ]:
              precision    recall  f1-score   support
         ham       0.95      1.00      0.97       965
        spam       0.96      0.67      0.79       150
    accuracy                           0.95      1115
   macro avg       0.96      0.83      0.88      1115
weighted avg       0.95      0.95      0.95      1115

In [ ]:
 test_model(df, fasttext, 'v3', 'v1')

'              precision    recall  f1-score   support\n         ham       0.94      0.98      0.96       965\n        spam       0.84      0.59      0.69       150\n    accuracy                           0.93      1115\n   macro avg       0.89      0.78      0.83      1115\nweighted avg       0.93      0.93      0.92      1115'

In [ ]:
              precision    recall  f1-score   support
         ham       0.94      0.98      0.96       965
        spam       0.84      0.59      0.69       150
    accuracy                           0.93      1115
   macro avg       0.89      0.78      0.83      1115
weighted avg       0.93      0.93      0.92      1115

In [ ]:
 test_model(df, fasttext, 'v4', 'v1')

'              precision    recall  f1-score   support\n         ham       0.94      0.99      0.96       965\n        spam       0.91      0.58      0.71       150\n    accuracy                           0.94      1115\n   macro avg       0.92      0.79      0.84      1115\nweighted avg       0.93      0.94      0.93      1115'

In [ ]:
              precision    recall  f1-score   support
         ham       0.94      0.99      0.96       965
        spam       0.91      0.58      0.71       150
    accuracy                           0.94      1115
   macro avg       0.92      0.79      0.84      1115
weighted avg       0.93      0.94      0.93      1115

 v2가 가장 높은 정확도를 보이며, 'spam'을 예측하는데 있어서도 상대적으로 높은 정밀도와 F1 점수를 나타낸다. v4는v3보다 정확도가 높고 spam에 대한 정밀도가 더 높지만 재현율이 다소 낮다.

---

스팸을 모두 걸러내면서 일부 정상 메일이 분류될 위험을 감수하고 싶다면, **재현율(Recall)**에 중점을 둔다. 반면에 중요한 정상 메일을 스팸으로 분류하는 것을 피하고 싶다면, **정밀도(Precision)**에 중점을 둔다.
 Word2Vec과 GloVe를 사용한 모델 v2와 v4는 스팸을 정확하게 분류하는데 효과적이다.그리고 토크나이저를 안한 v2가 좋다고 나오는데 이는 추가적인 데이터셋을 통한 검증이 필요하지 않을까 싶다.

---

##  심심풀이 CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# BOW 벡터로 변환
vectorizer = CountVectorizer(analyzer=lambda x: x)  # 이미 토큰화된 데이터 사용하기 위해 analyzer 지정
bow = vectorizer.fit_transform(df['v2'])

# One-hot Encoding 예시 출력
print(bow.toarray())


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
# 인코딩 (ham: 0, spam: 1)
df['v1'] = df['v1'].map({'ham': 0, 'spam': 1})

X_train, X_test, y_train, y_test = train_test_split(bow.toarray(), df['v1'], test_size=0.3, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Model accuracy: {accuracy}')


Model accuracy: 0.9772727272727273
